In [0]:
# Databricks notebook source
# MAGIC %pip install databricks-sdk --upgrade
# MAGIC dbutils.library.restartPython()

# COMMAND ----------

from databricks.sdk import WorkspaceClient
from databricks.sdk.service.catalog import MonitorInferenceLog, MonitorInferenceLogProblemType
from pyspark.sql import SparkSession

w = WorkspaceClient()

spark = SparkSession.builder.getOrCreate()

catalog_name = "jpmc_group_catalog"  # Replace with your Unity Catalog name
schema_name = "mlops"  # Replace with your schema name
table_name = "wine_predictions_v2_dabs"  # Replace with your inference table name

full_table_name = f"{catalog_name}.{schema_name}.{table_name}"
output_schema = f"{catalog_name}.{schema_name}"  # Schema where monitor metric tables will be created
assets_directory = "/Workspace/Users/owani@deloitte.com/databricks_lakehouse_monitoring"
# "/workspace/jpmc_group_catalog/mlops/databricks_lakehouse_monitoring"  # Directory for monitor assets

try:
    w.quality_monitors.delete(table_name=full_table_name)
except Exception:
    pass

spark.sql("""
    ALTER TABLE jpmc_group_catalog.mlops.wine_predictions_v2_dabs
    SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
""")

# Inference-specific parameters
problem_type = MonitorInferenceLogProblemType.PROBLEM_TYPE_REGRESSION  # Or CLASSIFICATION
prediction_column = "prediction"
timestamp_column = "timestamp"
model_id_column = "model_id"
label_column = "quality"  # Optional, only if ground truth labels are available
granularities = ["1 day"]  # Desired time windows for metric aggregation

In [0]:
# Code Generated by Sidekick is for learning and experimentation purposes only.
problem_type = MonitorInferenceLogProblemType.PROBLEM_TYPE_REGRESSION
prediction_column = "prediction"
timestamp_column = "timestamp"
model_id_column = "model_id"
label_column = "quality"  # omit if no ground truth
granularities = ["1 day"]

try:
    monitor = w.quality_monitors.create(
        table_name=full_table_name,
        output_schema_name=output_schema,
        assets_dir=assets_directory,
        inference_log=MonitorInferenceLog(
            problem_type=problem_type,
            prediction_col=prediction_column,
            timestamp_col=timestamp_column,
            model_id_col=model_id_column,
            label_col=label_column,
            granularities=granularities,  # <-- required in your SDK
        ),
        baseline_table_name="jpmc_group_catalog.mlops.wine_predictions_v2_dabs",
        # If your SDK *also* supports top-level granularities, you can keep it there,
        # but avoid duplicating if it errors:
        # granularities=granularities,
    )

    print(f"Inference Profile Monitor '{monitor}' created successfully.")
except Exception as e:
    print(f"Error creating monitor: {e}")
